In [1]:
import sys
sys.path.append('/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/RegionalExtremesPackage')

from main import parser_arguments, regional_extremes_method, local_extremes_method, regional_extremes_minicube

In [2]:
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client as daskClient

In [3]:
# Set distributed scheduler as default
import dask
dask.config.set(scheduler='distributed')

cluster = SLURMCluster(
    queue='work',                  # Specify the SLURM queue
    processes=1,                     # Number of processes per job
    cores=1,                          # Number of cores per job
    memory='50GB',                    # Memory per job
    walltime='15:00:00',              # Job duration (hh:mm:ss)
    job_script_prologue=[
        'module load BGC-easybuilded',
        'module load  GCC',
    ]
)

# Scale up the number of workers
#cluster.scale(jobs=20)  # Adjust the number of jobs/workers
cluster.adapt(minimum=0, maximum=30)

# Create a Dask client that connects to the cluster
client = daskClient(cluster)

# Check cluster status
cluster

/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38549 instead
  warnings.warn(


Dashboard: http://10.0.30.56:38549/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.0.30.56:33601,Workers: 0
Dashboard: http://10.0.30.56:38549/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
import sys
import argparse

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]


args = parser_arguments().parse_args()
args

Namespace(id=None, name=None, index='pei_180', compute_variance=False, region='globe', time_resolution=5, n_components=3, n_samples=100, n_eco_clusters=25, kernel_pca=False, saving_path=None, path_load_experiment=None, method='regional')

In [5]:
import numpy as np 

# Remove Jupyter's arguments before parsing your own
sys.argv = sys.argv[:1]
args = parser_arguments().parse_args()
args.name = "deep_extreme_global"
args.index = "EVI_EN"
args.k_pca = False
args.n_samples = 20000
args.n_components = 3
args.n_eco_clusters = 50
args.compute_variance = False
args.method = "regional"
args.start_year = 2000
args.is_generic_xarray_dataset = False

args.path_load_experiment = "/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2025-01-16_12:19:12_deep_extreme_global"

LOWER_QUANTILES_LEVEL = np.array([0.01, 0.025, 0.05])
UPPER_QUANTILES_LEVEL = np.array([0.95, 0.975, 0.99])
if args.method == "regional":
    # Apply the regional extremes method
    # regional_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
    regional_extremes_minicube(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "local":
    # Apply the uniform threshold method
    local_extremes_method(args, (LOWER_QUANTILES_LEVEL, UPPER_QUANTILES_LEVEL))
elif args.method == "global":
    raise NotImplementedError("the global method is not yet implemented.")


[2025-01-20 20:21:35] Loading of the model path: /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2025-01-16_12:19:12_deep_extreme_global
/Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2025-01-16_12:19:12_deep_extreme_global/EVI_EN
[2025-01-20 20:21:42] Projection loaded from /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2025-01-16_12:19:12_deep_extreme_global/EVI_EN/pca_projection_0.zarr
[2025-01-20 20:21:42] Limits eco_clusters loaded.
[2025-01-20 20:21:42] eco_clusters.zarr loaded.
[2025-01-20 20:21:42] thresholds.zarr loaded.
[2025-01-20 20:21:42] start of the preprocess
[2025-01-20 20:21:46] Computation on the entire dataset. 100 samples


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 26.29 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2025-01-20 20:24:34] msc computed and saved.


/Net/Groups/BGI/scratch/crobin/miniconda3/envs/ExtremesEnv2/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.29 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


[2025-01-20 20:24:44] returning time serie
[2025-01-20 20:24:44] Data are projected in the feature space.
[2025-01-20 20:24:49] PCA Projection computed and saved to /Net/Groups/BGI/scratch/crobin/PythonProjects/ExtremesProject/experiments/2025-01-16_12:19:12_deep_extreme_global/EVI_EN/mc_25.61_44.32_1.3_20231018_0/pca_projection_11.zarr.
[2025-01-20 20:25:13] eco_clusters computed and saved.
(array([0.01 , 0.025, 0.05 ]), array([0.95 , 0.975, 0.99 ]))
<xarray.Dataset> Size: 128B
Dimensions:          (quantile: 6)
Coordinates:
    quantile_levels  (quantile) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>
    cluster          object 8B (12, 5, 2)
    component_1      int64 8B 12
    component_2      int64 8B 5
    component_3      int64 8B 2
Dimensions without coordinates: quantile
Data variables:
    thresholds       (quantile) float64 48B dask.array<chunksize=(6,), meta=np.ndarray>
<xarray.DataArray 'quantile' (quantile: 6)> Size: 48B
array([0, 1, 2, 3, 4, 5])
Coordinates:
   

KeyError: "no index found for coordinate 'quantile_levels'"

In [1]:
client.close()
cluster.close()

NameError: name 'client' is not defined